# Jigsaw generation #1

In [1]:
import numpy as np

from tqdm import tqdm

from jigsaw import *
from jigsaw.default import *
from jigsaw.svg import *

In [2]:
generator = np.random.default_rng(42)

In [3]:
# Helper
def search(sampler):

    # Loop until a grid with a unique solution is found
    with tqdm() as progress:
        while True:

            # Sample grid, according to rules
            horizontal_edges, vertical_edges = sampler()
            pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    
            # Check for solution uniqueness
            is_unique = has_unique_solution(H, W, pieces, opposite, flip, constraints="border")
            progress.update(1)
            if is_unique:
                return horizontal_edges, vertical_edges

## Simple 3x3

In [4]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE'], dtype='<U22')

In [5]:
# Generate grid
H, W = 3, 3
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler)

3it [00:03,  1.05s/it]


In [6]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [7]:
# Keep it for later
pieces_3x3 = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Skewed 3x3

In [8]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['RIGHT_SMALL_MALE', 'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE',
       'LEFT_SMALL_FEMALE'], dtype='<U22')

In [9]:
# Generate grid
H, W = 3, 3
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler)

4it [00:00, 234.77it/s]


In [10]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [11]:
# Keep it for later
pieces_3x3_skewed = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Simple 4x4

In [12]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE'], dtype='<U22')

In [13]:
# Generate grid
H, W = 4, 4
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler)

61it [00:00, 175.50it/s]


In [14]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [15]:
# Keep it for later
pieces_4x4 = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Twisted 4x4

In [16]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    LEFT_SMALL_MALE,
    TWISTED_SMALL_MALE,
    DOUBLE_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['RIGHT_SMALL_MALE', 'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE',
       'LEFT_SMALL_FEMALE', 'DOUBLE_SMALL_MALE', 'DOUBLE_SMALL_FEMALE',
       'TWISTED_SMALL_MALE', 'TWISTED_SMALL_FEMALE'], dtype='<U22')

In [17]:
# Generate grid
H, W = 4, 4
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler)

2it [00:00, 133.26it/s]


In [18]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [19]:
# Keep it for later
pieces_4x4_twisted = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 4x4 with 8 corners

In [20]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [21]:
# Generate grid
H, W = 4, 4
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types)
    horizontal_edges[1:-1, 2] = FLAT
    vertical_edges[2, 1:-1] = FLAT
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler)

67it [00:00, 86.78it/s] 


In [22]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [23]:
# Keep it for later
pieces_4x4_cross = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 5x5 with square center

In [62]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE'], dtype='<U22')

In [ ]:
# Generate grid
H, W = 5, 5
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types)
    horizontal_edges[2, 2:-2] = FLAT
    vertical_edges[2:-2, 2] = FLAT
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler)

0it [00:00, ?it/s]

In [ ]:
display_grid(horizontal_edges, vertical_edges)

## 3x3 + 4x4 = 5x5

## Pack as a single grid

In [51]:
# Pack jigsaw puzzles as a single grid
pieces = np.zeros((7, 8, 4), dtype=np.uint8)
pieces[:4, :4] = pieces_4x4
pieces[:4, 4:8] = pieces_4x4_twisted
pieces[4:7, :3] = pieces_3x3
horizontal_edges, vertical_edges = pieces_to_grid(pieces, opposite)
display_grid(horizontal_edges, vertical_edges)

In [55]:
# Export as SVG
with open("attempt_01.svg", "w") as file:
    file.write(grid_to_svg(horizontal_edges, vertical_edges))